# WS24 - Intelligente Informationssysteme

## Block 3: Retrieval Augmented Generation

Build your first simple RAG with LangChain. We follow the LangChain Tutorial "Build a Retrieval Augmented Generation (RAG) App" found at <https://python.langchain.com/docs/tutorials/rag/>.

**Part 2: Retrieve Knowledge from Vectorstore and Generate Answers**



In [7]:
# Langchain
# Vector stores are commonly used for retrieval, but there are other ways to do retrieval, too.
# Retriever: An object that returns Documents given a text query
# Open a vectore store as retriever
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

from langchain_chroma import Chroma
vectorstore = Chroma(persist_directory="vector_store", collection_name="lils_blogs", embedding_function=embeddings)

vectorstore

In [29]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

question = "What are the approaches to task decomposition?"

retrieved_docs = retriever.invoke(question)

len(retrieved_docs)

6

In [30]:
print(retrieved_docs[0].page_content)

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.


In [33]:
## Chat Models: Modern LLMs are typically accessed through a chat model interface 
# that takes a list of messages as input and returns a message as output.
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:latest")

In [34]:
# LangChain Prompt
from langchain_core.prompts import PromptTemplate

prompt = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

prompt_template = PromptTemplate.from_template(prompt)

In [36]:
messages = prompt_template.invoke(
    {"context": retrieved_docs[0].page_content, "question": question}
).to_messages()
messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What are the approaches to task decomposition? \nContext: Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process. \nAnswer:", additional_kwargs={}, response_metadata={})]

In [40]:
message = llm.invoke(messages)

In [44]:
print(type(message))
print(message.content)
print("---------------")
print("id               :", message.id)
print("additional_kwargs:", message.additional_kwargs)
print("response_metadata:", message.response_metadata)
print("usage_metadata   :", message.usage_metadata)


<class 'langchain_core.messages.ai.AIMessage'>
There are at least two approaches to task decomposition mentioned in the context, but they are not explicitly listed as separate methods. One approach is the "Chain of Thought" (CoT) technique, where a model is instructed to break down complex tasks into smaller steps. Another approach involves planning and breaking down tasks into manageable parts for an agent.
---------------
id               : run-bd4776af-85b1-47aa-8a36-b82dae2e066a-0
additional_kwargs: {}
response_metadata: {'model': 'llama3.2:latest', 'created_at': '2024-11-27T04:29:19.817178Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1238415583, 'load_duration': 24195250, 'prompt_eval_count': 214, 'prompt_eval_duration': 178000000, 'eval_count': 69, 'eval_duration': 1034000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}
usage_metadata   : {'input_tokens': 214, 'output_tokens': 69, 'total_tokens': 283}
